In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from catboost import CatBoostRegressor
from sklearn.preprocessing import MinMaxScaler

import re
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import warnings

# Установка параметров для игнорирования предупреждений
warnings.filterwarnings('ignore')

pd.set_option('display.width', 200)


In [2]:
# Загрузка данных
data_train = pd.read_csv('data\\train.csv')
data_test = pd.read_csv('data\\test.csv')

In [3]:
data_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# Проверка пропусков по столбцам
missing_values = data_train.isnull().sum()

# Вывод количества пропусков по каждому столбцу
print("Количество пропусков по столбцам:")
print(missing_values)

Количество пропусков по столбцам:
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [5]:
# Проверка пропусков по столбцам
missing_values = data_test.isnull().sum()

# Вывод количества пропусков по каждому столбцу
print("Количество пропусков по столбцам:")
print(missing_values)

Количество пропусков по столбцам:
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [6]:
# Вручную заполняем пропущенные значения Embarked Fare
# Это не совсем правильно.         
data_train.loc[data_train['Name'].str.contains('Icard, Miss. Amelie'), 'Embarked'] = 'C'  # Шербур
data_train.loc[data_train['Name'].str.contains('Stone, Mrs. George Nelson'), 'Embarked'] = 'S'  # Саутгемптон
# Замена пропусков в 'Fare' на медиану
data_test['Fare'] = data_test['Fare'].fillna(data_test['Fare'].median())

In [7]:
# Удаляем столбец 'PassengerId'
data_train = data_train.drop(columns=['PassengerId'])

In [8]:
# Преобразуем значения в столбце 'Sex' (женщина = 1, мужчина = 0)
# Графики указали, что женщины имеют больший шанс на выживание
data_train['Sex'] = data_train['Sex'].map({'female': 1, 'male': 0}).astype("int8")
data_test['Sex'] = data_test['Sex'].map({'female': 1, 'male': 0}).astype("int8")

In [9]:
# Преобразуем значения в столбце 'Pclass' (3 -> 1, 1 -> 3)
# Первый класс имеет больше шансов, 3 меньше
data_train['Pclass'] = data_train['Pclass'].map({1: 3, 3: 1, 2: 2}).astype("int8")
data_test['Pclass'] = data_test['Pclass'].map({1: 3, 3: 1, 2: 2}).astype("int8")

In [10]:
# Применяем one-hot encoding с помощью pd.get_dummies
data_train = pd.get_dummies(data_train, columns=['Embarked'], drop_first=False, dtype='int8')
data_test = pd.get_dummies(data_test, columns=['Embarked'], drop_first=False, dtype='int8')

In [11]:
# Извлечение титула из строки
data_train['Title'] = data_train['Name'].str.extract(r'([A-Za-z]+)\.', expand=False)
data_test['Title'] = data_test['Name'].str.extract(r'([A-Za-z]+)\.', expand=False)

# Удаляем титул с точкой из строки
data_train['Name'] = data_train['Name'].str.replace(r'\s?[A-Za-z]+\.', '', regex=True).str.strip()
data_test['Name'] = data_test['Name'].str.replace(r'\s?[A-Za-z]+\.', '', regex=True).str.strip()

In [12]:
# Маппинг титулов
title_mapping = {
    'Mr': 'Mr',
    'Miss': 'Miss',
    'Mrs': 'Mrs',
    'Master': 'Master',
    'Dr': 'Rare',
    'Rev': 'Rare',
    'Mlle': 'Miss',
    'Major': 'Rare',
    'Col': 'Rare',
    'Countess': 'Noble',
    'Capt': 'Rare',
    'Ms': 'Miss',
    'Sir': 'Noble',
    'Lady': 'Noble',
    'Mme': 'Mrs',
    'Don': 'Noble',
    'Jonkheer': 'Noble'
}

# Применяем маппинг к столбцу 'Title'
data_train['TitleCategory'] = data_train['Title'].map(title_mapping).fillna('Other')
data_test['TitleCategory'] = data_test['Title'].map(title_mapping).fillna('Other')

In [13]:
# Применяем кластеризацию с 6 кластерами 
fare_data_train = data_train[['Fare']]
kmeans = KMeans(n_clusters=6, random_state=42)
data_train.loc[fare_data_train.index, 'FareCluster'] = kmeans.fit_predict(fare_data_train)
# Преобразуем столбец 'FareCluster' в тип int8
data_train['FareCluster'] = data_train['FareCluster'].astype('int8')

  File "C:\Users\14488\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\subprocess.py", line 1538, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(executable, args,
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [14]:
fare_data_test = data_test[['Fare']]
kmeans = KMeans(n_clusters=6, random_state=42)
data_test.loc[fare_data_test.index, 'FareCluster'] = kmeans.fit_predict(fare_data_test)
# Преобразуем столбец 'FareCluster' в тип int8
data_test['FareCluster'] = data_test['FareCluster'].astype('int8')

In [15]:
# Используем регулярные выражения для выделения префиксов и номеров билетов
data_train['Ticket_Prefix'] = data_train['Ticket'].str.extract(r'([A-Za-z\.\/\s]+)', expand=False)  # Выделяем префиксы
data_train['Ticket_Number'] = data_train['Ticket'].str.extract(r'(\d+)', expand=False)  # Выделяем номер билета
# Заменяем NaN в Ticket_Prefix на '0'
data_train['Ticket_Prefix'] = data_train['Ticket_Prefix'].fillna('0')
# Преобразуем Ticket_Number в числовой формат
data_train['Ticket_Number'] = pd.to_numeric(data_train['Ticket_Number'], errors='coerce')
# Заменяем NaN в Ticket_Number на 0
data_train['Ticket_Number'] = data_train['Ticket_Number'].fillna(0)
# Удаляем столбец 'Ticket'
data_train = data_train.drop(columns=['Ticket'])

In [16]:
# Используем регулярные выражения для выделения префиксов и номеров билетов
data_test['Ticket_Prefix'] = data_test['Ticket'].str.extract(r'([A-Za-z\.\/\s]+)', expand=False)  # Выделяем префиксы
data_test['Ticket_Number'] = data_test['Ticket'].str.extract(r'(\d+)', expand=False)  # Выделяем номер билета

# Заменяем NaN в Ticket_Prefix на '0'
data_test['Ticket_Prefix'] = data_test['Ticket_Prefix'].fillna('0')

# Преобразуем Ticket_Number в числовой формат
data_test['Ticket_Number'] = pd.to_numeric(data_test['Ticket_Number'], errors='coerce')

# Заменяем NaN в Ticket_Number на 0
data_test['Ticket_Number'] = data_test['Ticket_Number'].fillna(0)

# Удаляем столбец 'Ticket'
data_test = data_test.drop(columns=['Ticket'])


In [17]:
# Инициализируем LabelEncoder
encoder = LabelEncoder()

# Кодируем каждый столбец отдельно
data_train['Ticket_Prefix'] = encoder.fit_transform(data_train['Ticket_Prefix'])
data_train['Title'] = encoder.fit_transform(data_train['Title'])
data_train['Name'] = encoder.fit_transform(data_train['Name'])
data_train['TitleCategory'] = encoder.fit_transform(data_train['TitleCategory'])

In [18]:
# Инициализируем LabelEncoder
encoder = LabelEncoder()

# Кодируем каждый столбец отдельно
data_test['Ticket_Prefix'] = encoder.fit_transform(data_test['Ticket_Prefix'])
data_test['Title'] = encoder.fit_transform(data_test['Title'])
data_test['Name'] = encoder.fit_transform(data_test['Name'])
data_test['TitleCategory'] = encoder.fit_transform(data_test['TitleCategory'])


In [19]:
# Удалим ненужные столбцы
data_train_age = data_train.drop(columns=['Survived', 'Cabin'])

# Разделяем данные на два набора: те, где Age не пропущен, и те, где Age пропущен
data_train_not_na = data_train_age[data_train_age['Age'].notna()]
data_train_na = data_train_age[data_train_age['Age'].isna()]

# Определяем признаки (X) и целевую переменную (y) для обучения
X_train = data_train_not_na.drop(columns=['Age'])
y_train = data_train_not_na['Age']

# Масштабируем данные
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Строим модель CatBoost
catboost_model = CatBoostRegressor(iterations=1000, learning_rate=0.1, depth=6, random_seed=42, verbose=0)
catboost_model.fit(X_train_scaled, y_train)

# Для строк с пропущенными значениями в 'Age' предсказываем возраст
X_na = data_train_na.drop(columns=['Age'])
X_na_scaled = scaler.transform(X_na)
age_pred_catboost = catboost_model.predict(X_na_scaled)

# Заполняем пропуски в Age предсказанными значениями
data_train.loc[data_train['Age'].isna(), 'Age'] = age_pred_catboost


In [20]:
# Удалим ненужные столбцы
data_test_age = data_test.drop(columns=['Cabin', 'PassengerId'])

# Разделяем данные на два набора: те, где Age не пропущен, и те, где Age пропущен
data_test_not_na = data_test_age[data_test_age['Age'].notna()]
data_test_na = data_test_age[data_test_age['Age'].isna()]

# Определяем признаки (X) и целевую переменную (y) для обучения
X_train = data_test_not_na.drop(columns=['Age'])
y_train = data_test_not_na['Age']

# Масштабируем данные
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Строим модель CatBoost
catboost_model = CatBoostRegressor(iterations=1000, learning_rate=0.1, depth=6, random_seed=42, verbose=0)
catboost_model.fit(X_train_scaled, y_train)

# Для строк с пропущенными значениями в 'Age' предсказываем возраст
X_na = data_test_na.drop(columns=['Age'])
X_na_scaled = scaler.transform(X_na)
age_pred_catboost = catboost_model.predict(X_na_scaled)

# Заполняем пропуски в Age предсказанными значениями
data_test.loc[data_test['Age'].isna(), 'Age'] = age_pred_catboost


In [21]:
# Создаем признак количества кают
data_train['Cabin_Count'] = data_train['Cabin'].str.split().str.len().fillna(1).astype('int8')
data_test['Cabin_Count'] = data_test['Cabin'].str.split().str.len().fillna(1).astype('int8')

In [22]:
# Разделяем множественные каюты и берем первую
data_train['Primary_Cabin'] = data_train['Cabin'].str.split().str[0]
data_test['Primary_Cabin'] = data_test['Cabin'].str.split().str[0]

In [23]:
# Разделяем номер основной каюты на букву и число
data_train['Cabin_Letter'] = data_train['Primary_Cabin'].str[0]
data_test['Cabin_Letter'] = data_test['Primary_Cabin'].str[0]

In [24]:
data_train['Cabin_Number'] = data_train['Primary_Cabin'].str.extract('(\d+)').astype(float)
# Удаление столбца Primary_Cabin
data_train.drop(columns=['Primary_Cabin'], inplace=True)

data_test['Cabin_Number'] = data_test['Primary_Cabin'].str.extract('(\d+)').astype(float)
# Удаление столбца Primary_Cabin
data_test.drop(columns=['Primary_Cabin'], inplace=True)

In [25]:
# Выделяем необходимые признаки
features = ['Pclass', 'Sex', 'Age', 'Title', 'TitleCategory', 'Fare']
target = 'Cabin_Letter'

# Разделяем данные на обучающую и тестовую выборки
data_train_train = data_train[data_train[target].notna()]  # обучающая выборка (без NaN в Cabin_Letter)
data_train_test = data_train[data_train[target].isna()]   # тестовая выборка (с NaN в Cabin_Letter)

# Признаки и целевая переменная для обучения
X_train = data_train_train[features]
y_train = data_train_train[target]

# Масштабируем данные
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Признаки для теста (будем использовать их для предсказания)
X_test = data_train_test[features]
X_test_scaled = scaler.transform(X_test)

# Строим модель случайного леса
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# Оценка модели на обучающей выборке
y_pred_train = model.predict(X_train_scaled)

# Точность на обучающей выборке
accuracy = accuracy_score(y_train, y_pred_train)
print(f'Accuracy on training data_train: {accuracy:.4f}')

# Прогнозируем на тестовой выборке
y_pred_test = model.predict(X_test_scaled)

# Заполняем пропуски в 'Cabin_Letter' предсказанными значениями
data_train.loc[data_train[target].isna(), target] = y_pred_test

# Проверяем результат
print(data_train[target].isna().sum())  # проверим, сколько осталось пропусков в 'Cabin_Letter'


Accuracy on training data_train: 1.0000
0


In [26]:
# Выделяем необходимые признаки
features = ['Pclass', 'Sex', 'Age', 'Title', 'TitleCategory', 'Fare']
target = 'Cabin_Letter'

# Разделяем данные на обучающую и тестовую выборки
data_test_train = data_test[data_test[target].notna()]  # обучающая выборка (без NaN в Cabin_Letter)
data_test_test = data_test[data_test[target].isna()]   # тестовая выборка (с NaN в Cabin_Letter)

# Признаки и целевая переменная для обучения
X_train = data_test_train[features]
y_train = data_test_train[target]

# Масштабируем данные
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Признаки для теста (будем использовать их для предсказания)
X_test = data_test_test[features]
X_test_scaled = scaler.transform(X_test)

# Строим модель случайного леса
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# Оценка модели на обучающей выборке
y_pred_train = model.predict(X_train_scaled)

# Точность на обучающей выборке
accuracy = accuracy_score(y_train, y_pred_train)
print(f'Accuracy on training data_test: {accuracy:.4f}')

# Прогнозируем на тестовой выборке
y_pred_test = model.predict(X_test_scaled)

# Заполняем пропуски в 'Cabin_Letter' предсказанными значениями
data_test.loc[data_test[target].isna(), target] = y_pred_test

# Проверяем результат
print(data_test[target].isna().sum())  # проверим, сколько осталось пропусков в 'Cabin_Letter'


Accuracy on training data_test: 1.0000
0


In [27]:
# Теперь заполняем пропуски в 'Cabin_Number' медианными значениями для каждого 'Cabin_Letter'
for letter in data_train['Cabin_Letter'].unique():
    # Для каждой буквы находим медиану в 'Cabin_Number'
    median_value = data_train[data_train['Cabin_Letter'] == letter]['Cabin_Number'].median()
    
    # Заполняем пропуски в 'Cabin_Number' для соответствующей буквы
    data_train.loc[(data_train['Cabin_Letter'] == letter) & (data_train['Cabin_Number'].isna()), 'Cabin_Number'] = median_value

In [28]:
# Теперь заполняем пропуски в 'Cabin_Number' медианными значениями для каждого 'Cabin_Letter'
for letter in data_test['Cabin_Letter'].unique():
    # Для каждой буквы находим медиану в 'Cabin_Number'
    median_value = data_test[data_test['Cabin_Letter'] == letter]['Cabin_Number'].median()
    
    # Заполняем пропуски в 'Cabin_Number' для соответствующей буквы
    data_test.loc[(data_test['Cabin_Letter'] == letter) & (data_test['Cabin_Number'].isna()), 'Cabin_Number'] = median_value

In [29]:
data_train["Cabin_Number"] = data_train["Cabin_Number"].fillna(0)
data_train = data_train.drop(columns=["Cabin"])

data_test["Cabin_Number"] = data_test["Cabin_Number"].fillna(0)
data_test = data_test.drop(columns=["Cabin"])

In [30]:
# Кодируем титулы с помощью LabelEncoder
le = LabelEncoder()
data_train['Cabin_Letter'] = le.fit_transform(data_train['Cabin_Letter'])

# Кодируем титулы с помощью LabelEncoder
le = LabelEncoder()
data_test['Cabin_Letter'] = le.fit_transform(data_test['Cabin_Letter'])

In [31]:
# Создание нового признака FamilySize
data_train['FamilySize'] = data_train['SibSp'] + data_train['Parch'] + 1

# Признак одиночного пассажира
data_train['IsAlone'] = (data_train['FamilySize'] == 1).astype(np.int8)

# Стоимость билета на человека
data_train['FarePerPerson'] = data_train['Fare'] / data_train['FamilySize']

In [32]:
# Создание нового признака FamilySize
data_test['FamilySize'] = data_test['SibSp'] + data_test['Parch'] + 1

# Признак одиночного пассажира
data_test['IsAlone'] = (data_test['FamilySize'] == 1).astype(np.int8)

# Стоимость билета на человека
data_test['FarePerPerson'] = data_test['Fare'] / data_test['FamilySize']

In [33]:
data_test.head(5)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,...,TitleCategory,FareCluster,Ticket_Prefix,Ticket_Number,Cabin_Count,Cabin_Letter,Cabin_Number,FamilySize,IsAlone,FarePerPerson
0,892,1,206,0,34.5,0,0,7.8292,0,1,...,2,1,0,330911,1,5,4.0,1,1,7.829200
1,893,1,403,1,47.0,1,0,7.0000,0,0,...,3,1,0,363272,1,5,4.0,2,0,3.500000
2,894,2,269,0,62.0,0,0,9.6875,0,1,...,2,1,0,240276,1,5,4.0,1,1,9.687500
3,895,1,408,0,27.0,0,0,8.6625,0,0,...,2,1,0,315154,1,5,4.0,1,1,8.662500
4,896,1,178,1,22.0,1,1,12.2875,0,0,...,3,1,0,3101298,1,5,4.0,3,0,4.095833


In [34]:
# Записываем в файл trait_transform.csv
data_train.to_csv("data/train_transform.csv", index=False)

print("Данные успешно записаны в файл train_transform.csv")

Данные успешно записаны в файл train_transform.csv


In [35]:
# Записываем в файл trait_transform.csv
data_test.to_csv("data/test_transform.csv", index=False)

print("Данные успешно записаны в файл test_transform.csv")

Данные успешно записаны в файл test_transform.csv
